In [1]:
import os
import pandas as pd
import csv
import numpy as np
from stockstats import wrap

C:\Users\Elaine\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Elaine\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Elaine\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Get data from files

In [2]:
# get files
def files(dir):
    names = []
    for root, dirs, files in os.walk(dir):  
        for file in files:  
            if os.path.splitext(file)[1] == '.csv':  # csv files
                names.append(file)
    return names

In [3]:
# names for all files
names = files(os.getcwd() + "/data")

### Data for prediction models

In [4]:
for file in names:
    data = pd.read_csv("data/"+ file)
    y = data['Close']
    data = data.drop(['Currency', 'Close'], axis=1)
    df = wrap(data)

    # 80 metrics
    X = df[['low_2_sma', 'low_5_sma', 'low_10_sma', 'low_20_sma', 'low_50_sma', 'low_100_sma', 'low_2_ema', 'low_5_ema', 'low_10_ema', 'low_20_ema','low_50_ema', 'low_100_ema', 'low_5_mvar', 'low_20_mvar', 'low_50_mvar', 'low_100_mvar', 'low_5_mstd', 'low_20_mstd', 'low_50_mstd', 'low_100_mstd',
'high_2_sma', 'high_5_sma', 'high_10_sma', 'high_20_sma', 'high_50_sma', 'high_100_sma', 'high_2_ema', 'high_5_ema', 'high_10_ema', 'high_20_ema','high_50_ema', 'high_100_ema', 'high_5_mvar', 'high_20_mvar', 'high_50_mvar', 'high_100_mvar', 'high_5_mstd', 'high_20_mstd', 'high_50_mstd', 'high_100_mstd',
'low_2_sma', 'low_5_sma', 'low_10_sma', 'low_20_sma', 'low_50_sma', 'low_100_sma', 'low_2_ema', 'low_5_ema', 'low_10_ema', 'low_20_ema','low_50_ema', 'low_100_ema', 'low_5_mvar', 'low_20_mvar', 'low_50_mvar', 'low_100_mvar', 'low_5_mstd', 'low_20_mstd', 'low_50_mstd', 'low_100_mstd',
'volume_2_sma', 'volume_5_sma', 'volume_10_sma', 'volume_20_sma', 'volume_50_sma', 'volume_100_sma', 'volume_2_ema', 'volume_5_ema', 'volume_10_ema', 'volume_20_ema','volume_50_ema', 'volume_100_ema', 'volume_5_mvar', 'volume_20_mvar', 'volume_50_mvar', 'volume_100_mvar', 'volume_5_mstd', 'volume_20_mstd', 'volume_50_mstd', 'volume_100_mstd'
]]
    y = pd.DataFrame(y)
    y_change = y.pct_change()
    X = X.reset_index()
    df = X.join(y_change)
    df = df.iloc[: , 1:]
    newest_date = df.iloc[-1:].copy()
    df.drop(df.tail(1).index,inplace=True)
    df.dropna()

    # to csv
    df.to_csv("ml/" + file,index = False)

In [8]:
# for each cryptocurrency, we generate a csv like this
df

,low_2_sma,low_5_sma,low_10_sma,low_20_sma,low_50_sma,low_100_sma,low_2_ema,low_5_ema,low_10_ema,low_20_ema,...,volume_100_ema,volume_5_mvar,volume_20_mvar,volume_50_mvar,volume_100_mvar,volume_5_mstd,volume_20_mstd,volume_50_mstd,volume_100_mstd,Close
0,0.135573,0.135573,0.135573,0.135573,0.135573,0.135573,0.135573,0.135573,0.135573,0.135573,...,4.921770e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.113455,0.113455,0.113455,0.113455,0.113455,0.113455,0.102396,0.109031,0.111243,0.112349,...,9.361897e+06,3.865253e+13,3.865253e+13,3.865253e+13,3.865253e+13,6.217116e+06,6.217116e+06,6.217116e+06,6.217116e+06,-0.122494
2,0.091476,0.106175,0.106175,0.106175,0.106175,0.106175,0.094932,0.100781,0.103352,0.104735,...,8.887724e+06,1.993428e+13,1.993428e+13,1.993428e+13,1.993428e+13,4.464783e+06,4.464783e+06,4.464783e+06,4.464783e+06,-0.154128
3,0.093791,0.103623,0.103623,0.103623,0.103623,0.103623,0.095631,0.098782,0.100920,0.102204,...,1.198424e+07,4.954772e+13,4.954772e+13,4.954772e+13,4.954772e+13,7.039014e+06,7.039014e+06,7.039014e+06,7.039014e+06,-0.036444
4,0.086963,0.098490,0.098490,0.098490,0.098490,0.098490,0.083800,0.090788,0.094328,0.096339,...,1.207940e+07,3.722421e+13,3.722421e+13,3.722421e+13,3.722421e+13,6.101165e+06,6.101165e+06,6.101165e+06,6.101165e+06,-0.082470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,0.043735,0.045095,0.044599,0.043425,0.040205,0.042240,0.043427,0.044326,0.044279,0.043302,...,1.815607e+09,1.480986e+15,3.743598e+15,4.594331e+15,2.225558e+16,3.848359e+07,6.118495e+07,6.778150e+07,1.491830e+08,-0.049242
1666,0.042078,0.043999,0.044238,0.043440,0.040261,0.042046,0.042168,0.043397,0.043781,0.043134,...,1.782232e+09,1.843378e+15,3.801537e+15,4.676049e+15,1.795711e+16,4.293458e+07,6.165661e+07,6.838164e+07,1.340042e+08,-0.032728
1667,0.039118,0.042168,0.043654,0.043182,0.040288,0.041917,0.038521,0.041164,0.042493,0.042521,...,1.751033e+09,2.069864e+15,3.422598e+15,4.650990e+15,1.209960e+16,4.549576e+07,5.850298e+07,6.819817e+07,1.099982e+08,-0.110651
1668,0.036516,0.040408,0.043089,0.042900,0.040268,0.041875,0.037063,0.039554,0.041373,0.041932,...,1.718574e+09,1.492511e+15,3.509679e+15,4.466227e+15,1.023384e+16,3.863303e+07,5.924254e+07,6.682984e+07,1.011624e+08,0.009335


### Data for Optimization

In [5]:
# short-term: 1 day
date = "2021-08-28"

with open("op_data_1.csv",'a', newline = "") as f:
    csv_write = csv.writer(f)
    csv_write.writerow(["Name", "Open", "Close"])

    for file in names:
        content = pd.read_csv("data/"+ file)
        name = file.split(".")[0]
        open_price = content[content['Date'] == date].to_numpy()[0,1]
        close_price = content[content['Date'] == date].to_numpy()[0,4]
        csv_write.writerow([name, open_price, close_price])

In [6]:
# long-term: 1 month
start_date = "2021-07-28"
end_date = "2021-08-28"

with open("op_data_2.csv",'a', newline = "") as f:
    csv_write = csv.writer(f)
    csv_write.writerow(["Name", "Open", "Close", "Risk"])

    for file in names:
        content = pd.read_csv("data/"+ file)
        name = file.split(".")[0]
        open_price = content[content['Date'] == start_date].to_numpy()[0,1]
        close_price = content[content['Date'] == end_date].to_numpy()[0,4]
        
        # calculate variance
        close = content[(content['Date'] >= start_date) & (content['Date'] <= end_date)].to_numpy()[:,4]
        risk = np.var(close, ddof = 1)
        
        csv_write.writerow([name, open_price, close_price, risk])

In [7]:
# 1 month time series
start_date = "2021-07-28"
end_date = "2021-08-28"

# initialize
content = pd.read_csv("data/"+ names[0])
name = names[0].split(".")[0]

close = content[(content['Date'] >= start_date) & (content['Date'] <= end_date)][["Date", "Close"]]
total = close
total.rename(columns = {'Close': name}, inplace = True)

for file in names[1:]:
    content = pd.read_csv("data/"+ file)
    name = file.split(".")[0]

    close = content[(content['Date'] >= start_date) & (content['Date'] <= end_date)].to_numpy()[:,4]
    total[name] = close

    total.to_csv("op_data_3.csv",index = False)